In [44]:
import pandas as pd
import numpy as np
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re

In [45]:
df = pd.read_csv('actor_6.csv')

In [46]:
df.head(5)

,ids,times,stars,previews,likes,pos_p,month,hour,郭敬明,赵薇,陈凯歌,尔冬升,李诚儒,评价
0,横道世之介,2020-10-02 23:57:36,20,看这个节目最大的舒适感，就是屏蔽完赵薇郭敬明呱噪的业务指点，还能继续有幸听凯歌导演儒雅的，不...,515,0.5475,10,23,1.0,1.0,0.0,0.0,0.0,1.0
1,海边的西米,2020-10-03 02:37:43,40,想看尔冬升骂遍所有想露脸十八线男团idol,2623,0.4048,10,2,0.0,0.0,0.0,1.0,0.0,5.0
2,kakakarl,2020-10-02 23:09:01,30,太奇怪了，算上郭敬明，四个评委导演也都是拍电影的，而制片人评级都是用电视剧的标准在看演员，太...,895,0.0044,10,23,1.0,0.0,0.0,1.0,0.0,3.0
3,一口大井子,2020-10-12 01:06:51,20,陈凯歌最像学院老师，四个人里最会调教演员的。尔冬升是艺考的考官，严厉但准确。赵薇像辅导员，当...,341,0.0663,10,1,1.0,1.0,1.0,1.0,0.0,1.0
4,DING,2020-10-03 12:12:00,40,太喜欢温峥嵘那种坐在B级却浑身上下散发出S级光芒的状态，好演员值得被看见。,1287,0.9747,10,12,0.0,0.0,0.0,0.0,0.0,5.0


In [47]:
#构建柱状图---评论词导演热度排名
df.iloc[:,8:12].head(5)

,郭敬明,赵薇,陈凯歌,尔冬升
0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0
3,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0


In [48]:
df.iloc[:,8:12].sum()

郭敬明    166.0
赵薇      33.0
陈凯歌     84.0
尔冬升     71.0
dtype: float64

In [49]:
df.iloc[:,8:12].sum().reset_index()

,index,0
0,郭敬明,166.0
1,赵薇,33.0
2,陈凯歌,84.0
3,尔冬升,71.0


In [50]:
dom = df.iloc[:,8:12].sum().reset_index().sort_values(0,ascending = False)

In [51]:
dom

,index,0
0,郭敬明,166.0
2,陈凯歌,84.0
3,尔冬升,71.0
1,赵薇,33.0


In [52]:
dom.columns = ['角色','次数']#添加列，这上面的index和0不是列名称，所以不能重命名 dom['角色','次数'] = dom['index','0']会报错

In [53]:
dom['占比'] = round(dom['次数']/dom['次数'].sum(),2)*100

In [54]:
dom

,角色,次数,占比
0,郭敬明,166.0,47.0
2,陈凯歌,84.0,24.0
3,尔冬升,71.0,20.0
1,赵薇,33.0,9.0


In [55]:
dom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 1
Data columns (total 3 columns):
角色    4 non-null object
次数    4 non-null float64
占比    4 non-null float64
dtypes: float64(2), object(1)
memory usage: 128.0+ bytes


In [56]:
#绘制一个双y轴，一个是bar，一个是line的组合图，这里我们需要用overlap()

In [57]:
#example
from pyecharts.faker import Faker

In [58]:
x = list(dom['角色'])
y1 = list(dom['次数'])
y2 = list(dom['占比'])

In [59]:
x,y1,y2

(['郭敬明', '陈凯歌', '尔冬升', '赵薇'],
 [166.0, 84.0, 71.0, 33.0],
 [47.0, 24.0, 20.0, 9.0])

In [60]:
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))  # 设置主题
    .add_xaxis(x)
    .add_yaxis("次数", y1,bar_width = '50%',is_legend_hover_link = True,)
    # .add_yaxis("占比", y2)
    .extend_axis(
        yaxis=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} %"), interval=10
        )
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True,position = "inside",),
    markline_opts=opts.MarkLineOpts(
                data=[
                    # opts.MarkLineItem(type_="min", name="最小值"),
                    opts.MarkLineItem(type_="max", name="最大值")
                ]
            )
)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="导演热度排名及占比"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} 次"),max_= 200,interval = 40),
    )
)

line = Line().add_xaxis(x).add_yaxis("占比", y2, yaxis_index=1)
bar.overlap(line)
bar.render_notebook()

又学习了一遍关于配置项的东西，还是那句老话，温故而知新，可以为师矣

In [61]:
#对0进行处理
hourlist = []
for i in df['hour']:
    if i == 0:
        i = 24
    #print(i)
    hourlist.append(i)
# print(hourlist)
df['hour'] = hourlist

In [62]:
#对观看时间进行分析
place_message = df.groupby(['hour'])
place_com = place_message['hour'].agg(['count'])
place_com.reset_index(inplace=True)
place_com_last = place_com.sort_index()
dom1 = place_com_last.sort_values('hour', ascending=True)

In [63]:
dom1

,hour,count
0,1,36
1,2,15
2,3,13
3,4,7
4,5,8
5,6,3
6,7,6
7,8,13
8,9,23
9,10,15


In [64]:
x_h = list(dom1['hour'])
#len(list(dom1['hour'])) 我们需要将0改成24，有两种办法
# x_hlist = []
# x_hlist.extend([i for i in range(1,25)])
y_h = list(dom1['count'])

In [65]:
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.CHALK))  # 设置主题
    .add_xaxis(x_h)
    .add_yaxis("次数", y_h,bar_width = '50%',is_legend_hover_link = True,)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True),
    markline_opts=opts.MarkLineOpts(
                data=[
                    opts.MarkLineItem(type_="max", name="最大值")
                ]
            )
)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="评论时间分布"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} 次"),interval = 10),
    )
)
bar.render_notebook()

In [66]:
(ds1,ds2,ds3,ds4,ds5) = ([],[],[],[],[])
# for i in df['stars']:
#     # print(i)
#     if i == 10:
#         ds1.append(i)
#     if i == 20:
#         ds2.append(i)
#     if i == 30:
#         ds3.append(i)
#     if i == 40:
#         ds4.append(i)
#     if i == 50:
#         ds5.append(i)
place_message = df.groupby(['stars'])
place_com = place_message['stars'].agg(['count'])
place_com.reset_index(inplace=True)
place_com_last = place_com.sort_index()
dom3 = place_com_last.sort_values('stars', ascending=True)

In [67]:
dom3

,stars,count
0,10,259
1,20,101
2,30,80
3,40,38
4,50,11


In [68]:
x_s = ['一星','二星','三星','四星','五星']
# y_s = [len(ds1),len(ds2),len(ds3),len(ds4),len(ds5)]
y_s = list(dom3['count'])

In [69]:
x_s,y_s

(['一星', '二星', '三星', '四星', '五星'], [259, 101, 80, 38, 11])

In [70]:
pie = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(x_s, y_s)],
        radius=["30%", "55%"],
        center=["75%", "50%"],
        rosetype="area",
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="各星级分布"))
)
pie.render_notebook()

In [71]:
dom2 = df.iloc[:,2:5]   ##有个问题，如何任意取出两列，还未解决

In [72]:
df['likes'].loc[df['stars'] == 10].sum()

4217

In [73]:
# (dt1,dt2,dt3,dt4,dt5) = (0,0,0,0,0)
(dt1,dt2,dt3,dt4,dt5) = ([],[],[],[],[])

In [74]:
# for i in range(len(list(df['stars']))):
#     if list(df['stars'])[i] == 10:
#         dt1 += list(df['likes'])[i]
#     if list(df['stars'])[i] == 20:
#         dt2 += list(df['likes'])[i] 
#     if list(df['stars'])[i] == 30:
#         dt3 += list(df['likes'])[i]
#     if list(df['stars'])[i] == 40:
#         dt4 += list(df['likes'])[i] 
#     if list(df['stars'])[i] == 50:
#         dt5 += list(df['likes'])[i] 
dt1 = int(df['likes'].loc[df['stars'] == 10].sum())#按照行去求和
dt2 = int(df['likes'].loc[df['stars'] == 20].sum())
dt3 = int(df['likes'].loc[df['stars'] == 30].sum())
dt4 = int(df['likes'].loc[df['stars'] == 40].sum())
dt5 = int(df['likes'].loc[df['stars'] == 50].sum())

In [75]:
#type(dt1)

In [76]:
x_t = ['一星','二星','三星','四星','五星']
y_t = [dt1,dt2,dt3,dt4,dt5]

In [77]:
pie = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(x_t, y_t)],
        radius=["40%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="各星级点赞数分布"))
)
pie.render_notebook()

In [78]:
# y_t

In [79]:
# df['likes'].sort_values(0,ascending = False)

In [80]:
x_p = []
dom5 = []
for i in range(1,25):
    dp = round(float(df['pos_p'].loc[df['hour'] == i].sum()),2)#这里应该是分析一天的情感  /(df['pos_p'].loc[df['hour']].sum()/5
    x_p.append(str(i))#注意折线图的x必须是str，要不然机会错位或者图形
    dom5.append(dp)

In [81]:
# dom5

In [99]:
line = (
    Line(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC))  # 设置主题
    .add_xaxis(x_p)
    .add_yaxis("情感值", dom5,
    markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值"),
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(type_="average", name="平均值")]
        ),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True,position = 'left',)
)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="情感值时间分布"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} ")),
    )
)
line.render_notebook()

In [41]:
#应该统计某一天的
